In [1]:
!nvidia-smi


Fri Apr  5 02:19:51 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.65                 Driver Version: 551.86         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...    On  |   00000000:01:00.0  On |                  N/A |
| N/A   48C    P8              4W /  114W |     706MiB /   8188MiB |      7%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

from cnn import CNN

In [3]:
print(torch.cuda.get_device_name())
print(torch.__version__)
print(torch.version.cuda)
x = torch.randn(1).cuda()
print(x)

NVIDIA GeForce RTX 4070 Laptop GPU
2.2.2+cu121
12.1
tensor([-0.8227], device='cuda:0')


In [4]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    ])

train_dataset = datasets.ImageFolder(root='./data/transformed', transform=transform)

In [5]:
X = []  # to store the image tensors
y = []  # to store the labels

for img, label in train_dataset:
    X.append(img)
    y.append(label)

# Convert the lists to a tensor and a LongTensor, respectively, if needed
X = torch.stack(X)
y = torch.LongTensor(y)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [8]:
#train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)


In [9]:
model = CNN(16)
model.to(device)

hello world


CNN(
  (feature_extractor): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): Dropout(p=0.5, inplace=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(128, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): Dropout(p=0.5, inplace=False)
    (12): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(512, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (

In [31]:
model.train_model(X_train, y_train)

Epoch: 1/50 | Loss: 2.162205696105957
Epoch: 2/50 | Loss: 2.1532626152038574
Epoch: 3/50 | Loss: 2.0746357440948486
Epoch: 4/50 | Loss: 2.1022379398345947
Epoch: 5/50 | Loss: 1.9333223104476929
Epoch: 6/50 | Loss: 1.9891912937164307
Epoch: 7/50 | Loss: 2.0935170650482178
Epoch: 8/50 | Loss: 2.0927276611328125
Epoch: 9/50 | Loss: 2.030424118041992
Epoch: 10/50 | Loss: 1.9370970726013184
Epoch: 11/50 | Loss: 1.9664620161056519
Epoch: 12/50 | Loss: 2.0935192108154297
Epoch: 13/50 | Loss: 2.0009677410125732
Epoch: 14/50 | Loss: 2.081650733947754
Epoch: 15/50 | Loss: 2.030170440673828
Epoch: 16/50 | Loss: 2.0308444499969482
Epoch: 17/50 | Loss: 2.093583106994629
Epoch: 18/50 | Loss: 2.030856132507324
Epoch: 19/50 | Loss: 1.9984254837036133
Epoch: 20/50 | Loss: 2.1245276927948
Epoch: 21/50 | Loss: 2.1870927810668945
Epoch: 22/50 | Loss: 2.0095741748809814
Epoch: 23/50 | Loss: 1.9714547395706177
Epoch: 24/50 | Loss: 2.0620920658111572
Epoch: 25/50 | Loss: 1.9683502912521362
Epoch: 26/50 | Los

In [32]:
model.to('cpu')
torch.save(model,'./model')

In [33]:
loaded_model = torch.load('./model')

In [ ]:
#test_dataset = datasets.ImageFolder(root='./data/test_images', transform=transform)
#test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [34]:
correct = 0
total = 0
loaded_model.to(device)

CNN(
  (feature_extractor): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): Dropout(p=0.5, inplace=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(128, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): Dropout(p=0.5, inplace=False)
    (12): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(512, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (

In [36]:
y_pred = loaded_model.predict(X_test) # Only Test 

In [52]:
((torch.argmax(y_pred,1)==y_test).sum().item())/len(y_test)

0.6847916666666667

In [ ]:
# with torch.no_grad():
#     for X_test, y_test in test_loader:
#         X_test = X_test.to(device)
#         y_test = y_test.to(device)

#         outputs = loaded_model(X_test)
#         _, predicted = torch.max(outputs.data, 1)

#         total += y_test.size(0)

#         correct += (predicted == y_test).sum().item()


# accuracy = 100 * correct / total
# print(f'Accuracy of the CNN on the test data is: {accuracy:.2f}%')

In [ ]:
from PIL import Image

In [ ]:
test_image_path = 'full_Images\B\B1532.jpg'
test_image = Image.open(test_image_path)
test_image

In [ ]:
input_tensor = transform(test_image).unsqueeze(0)
input_tensor = input_tensor.to(device)
with torch.no_grad():
    output = loaded_model(input_tensor)


softmax = torch.nn.Softmax(dim=1)
probabilities = softmax(output)
output_numpy = output.cpu().numpy()
print(output_numpy)

In [ ]:
probabilities = torch.nn.functional.softmax(output, dim=1)
predicted_class = torch.argmax(probabilities, dim=1).item()
print(f"The predicted letter is: {predicted_class}")